#### Alguien le contó al cliente que el uso de withColumn y withColumnRenamed no correspondian a las best-practices dentro del desarrollo con Apache Spark, por lo tanto ha solicitado que hagan las transformaciones utilizando únicamente instrucciones de tipo Select. Adicional nos ha solicitado agregar algunas operaciones de limpieza no contempladas anteriormente. Presta mucha atención a cada requerimiento solicitado.

#### Por si fuera poco nos ha solicitado filtrar algunos datos que no le serán de utilidad y agregar una columna adicional al dataframe movies_df donde requiere el uso de expresiones regulares (por suerte un especialista nos ha dado las expresiones regulares a utilizar). No estaría mal estudiar un poco sobre expresiones regulares si consideras que no dominas el tema. 

##### Nota: Para poder trabajar con este notebook es necesario haber terminado el ejercicio de la sesión 04

In [1]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
%run utils.ipynb

In [18]:
# NO MODIFICAR CONTENIDO DE ESTA CELDA
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.types as t

# Creación de sesión de Spark
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("ejercicio_6") \
    .getOrCreate()

# Carga de tablas requeridas
root_path = "../resources/data/tmp/parquet/"
names_list = ["04/movies", "04/ratings", "04/tags"]
df_dict = read_tmp_df(names_list)

movies_df = df_dict["04/movies"]
ratings_df = df_dict["04/ratings"]
tags_df = df_dict["04/tags"]

movies_df.show(1, False)
ratings_df.show(1)
tags_df.show(1)

+-------+----------------+-------------------------------------------+
|movieId|title           |genres                                     |
+-------+----------------+-------------------------------------------+
|1      |Toy Story (1995)|Adventure|Animation|Children|Comedy|Fantasy|
+-------+----------------+-------------------------------------------+
only showing top 1 row

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|      1|   4.0|1225734739|
+------+-------+------+----------+
only showing top 1 row

+------+-------+------+----------+
|userId|movieId|   tag| timestamp|
+------+-------+------+----------+
|224183|    832|acting|1496668827|
+------+-------+------+----------+
only showing top 1 row



#### Actividad 1:
##### TO DO ->    Para el dataframe "movies_df":
- ##### El archivo contiene espacios a la izquiera y derecha en la columna "title". Elimina estos espacios.
    - Apoyate de la funcion trim -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.trim.html#pyspark.sql.functions.trim
- ##### Convierte la columna "genres" en un array, donde cada genero corresponde a una posición del array generado. Como resultado el esquema para la columna "genres" será un ArrayType(StringType()). 
    - Apoyate de la función split de Spark -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.split.html#pyspark.sql.functions.split
- ##### Renombra la columna "movieId" por "movie_id"

##### NO UTILIZAR withColumn NI withColumnRenamed

In [19]:
## TU CODIGO VA EN ESTA CELDA:
casted_movies_df = movies_df # aplicar transformaciones a movies_df

# Importaciones extra
from pyspark.sql.functions import trim
from pyspark.sql.functions import split
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType


# Eliminar espacios en la columna de "title"
casted_movies_df = casted_movies_df.select("movieId",trim(casted_movies_df["title"]).alias("title"), "genres")

# Convertir la columna "genres" en un array
casted_movies_df = casted_movies_df.select("movieId", "title", split(casted_movies_df["genres"], "\\|").cast(ArrayType(StringType())).alias("genres"))

# Renombrar la columna "movieId" por "movie_id"
casted_movies_df = casted_movies_df.select(casted_movies_df["movieId"].alias("movie_id"), "title", "genres")

# Imprimir resultados
casted_movies_df.show()

+--------+--------------------+--------------------+
|movie_id|               title|              genres|
+--------+--------------------+--------------------+
|       1|    Toy Story (1995)|[Adventure, Anima...|
|       2|      Jumanji (1995)|[Adventure, Child...|
|       3|Grumpier Old Men ...|   [Comedy, Romance]|
|       4|Waiting to Exhale...|[Comedy, Drama, R...|
|       5|Father of the Bri...|            [Comedy]|
|       6|         Heat (1995)|[Action, Crime, T...|
|       7|      Sabrina (1995)|   [Comedy, Romance]|
|       8| Tom and Huck (1995)|[Adventure, Child...|
|       9| Sudden Death (1995)|            [Action]|
|      10|    GoldenEye (1995)|[Action, Adventur...|
|      11|American Presiden...|[Comedy, Drama, R...|
|      12|Dracula: Dead and...|    [Comedy, Horror]|
|      13|        Balto (1995)|[Adventure, Anima...|
|      14|        Nixon (1995)|             [Drama]|
|      15|Cutthroat Island ...|[Action, Adventur...|
|      16|       Casino (1995)|      [Crime, D

In [20]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
casted_movies_df.show(1, False)
"""
Ejemplo de salida esperada:
+----------------+-------------------------------------------------+--------+
|title           |genres                                           |movie_id|
+----------------+-------------------------------------------------+--------+
|Toy Story (1995)|[Adventure, Animation, Children, Comedy, Fantasy]|1       |
+----------------+-------------------------------------------------+--------+
only showing top 1 row
"""

+--------+----------------+-------------------------------------------------+
|movie_id|title           |genres                                           |
+--------+----------------+-------------------------------------------------+
|1       |Toy Story (1995)|[Adventure, Animation, Children, Comedy, Fantasy]|
+--------+----------------+-------------------------------------------------+
only showing top 1 row



'\nEjemplo de salida esperada:\n+----------------+-------------------------------------------------+--------+\n|title           |genres                                           |movie_id|\n+----------------+-------------------------------------------------+--------+\n|Toy Story (1995)|[Adventure, Animation, Children, Comedy, Fantasy]|1       |\n+----------------+-------------------------------------------------+--------+\nonly showing top 1 row\n'

In [21]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
assert "movie_id" in casted_movies_df.columns
assert "title" in casted_movies_df.columns
assert "genres" in casted_movies_df.columns
assert len(casted_movies_df.columns) == 3

casted_movies_df = casted_movies_df.select("movie_id", "title", "genres")
assert schema_to_ddl(casted_movies_df) == 'movie_id STRING,title STRING,genres ARRAY<STRING>'

#### Actividad 2:
##### TO DO ->    Para el dataframe "ratings_df":
- ##### Renombra la columna "movieId" por "movie_id"
- ##### Renombra la columna "userId" por "user_id"
- ##### Castea la columna "rating" a formato double.
- ##### Convierte la columna "timestamp" a formato TimeStampType con formato logico yyyy-MM-dd HH:mm:ss, la nueva columna generada será "time". 
    - Utiliza la función: timestamp_seconds de Spark -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.timestamp_seconds.html#pyspark.sql.functions.timestamp_seconds
- ##### Elimina la columna "timestamp"
##### NO UTILIZAR withColumn NI withColumnRenamed

In [46]:
## TU CODIGO VA EN ESTA CELDA:
casted_ratings_df = ratings_df # aplicar transformaciones a ratings_df

# Importaciones extra
from pyspark.sql.functions import col, timestamp_seconds
from pyspark.sql.types import DoubleType, LongType, TimestampType

# Renombrar las columnas y cambio de formatos (Double y TimeStampType)
casted_ratings_df = casted_ratings_df.select(
    col("movieId").alias("movie_id"),
    col("userId").alias("user_id"),
    col("rating").cast(DoubleType()).alias("rating"),
    (timestamp_seconds(col("timestamp").cast(LongType())).alias("time"))
)

# Eliminar columna "timestamp"
casted_ratings_df = casted_ratings_df.drop("timestamp")

#Imprimir resultados
casted_ratings_df.show()

+--------+-------+------+-------------------+
|movie_id|user_id|rating|               time|
+--------+-------+------+-------------------+
|       1|      1|   4.0|2008-11-03 11:52:19|
|     110|      1|   4.0|2008-11-05 00:04:46|
|     158|      1|   4.0|2008-11-03 11:31:43|
|     260|      1|   4.5|2008-11-03 12:00:04|
|     356|      1|   5.0|2008-11-03 11:58:39|
|     381|      1|   3.5|2008-11-03 11:41:45|
|     596|      1|   4.0|2008-11-03 11:32:04|
|    1036|      1|   5.0|2008-11-03 12:07:06|
|    1049|      1|   3.0|2008-11-03 11:41:19|
|    1066|      1|   4.0|2008-11-03 12:29:21|
|    1196|      1|   3.5|2008-11-03 12:04:01|
|    1200|      1|   3.5|2008-11-03 12:11:01|
|    1210|      1|   4.5|2008-11-03 12:00:10|
|    1214|      1|   4.0|2008-11-03 12:20:26|
|    1291|      1|   5.0|2008-11-03 11:53:29|
|    1293|      1|   2.0|2008-11-03 11:37:22|
|    1376|      1|   3.0|2008-11-03 11:32:19|
|    1396|      1|   3.0|2008-11-03 11:32:14|
|    1537|      1|   4.0|2008-11-0

In [47]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
casted_ratings_df.show(1)
"""
Ejemplo de salida esperada:
+-------+--------+------+-------------------+
|user_id|movie_id|rating|               time|
+-------+--------+------+-------------------+
|      1|       1|   4.0|2008-11-03 11:52:19|
+-------+--------+------+-------------------+
only showing top 1 row
"""

+--------+-------+------+-------------------+
|movie_id|user_id|rating|               time|
+--------+-------+------+-------------------+
|       1|      1|   4.0|2008-11-03 11:52:19|
+--------+-------+------+-------------------+
only showing top 1 row



'\nEjemplo de salida esperada:\n+-------+--------+------+-------------------+\n|user_id|movie_id|rating|               time|\n+-------+--------+------+-------------------+\n|      1|       1|   4.0|2008-11-03 11:52:19|\n+-------+--------+------+-------------------+\nonly showing top 1 row\n'

In [48]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
assert "user_id" in casted_ratings_df.columns
assert "movie_id" in casted_ratings_df.columns
assert "rating" in casted_ratings_df.columns
assert "time" in casted_ratings_df.columns
assert len(casted_ratings_df.columns) == 4

casted_ratings_df = casted_ratings_df.select("user_id" ,"movie_id", "rating", "time")

assert schema_to_ddl(casted_ratings_df) == 'user_id STRING,movie_id STRING,rating DOUBLE,time TIMESTAMP'

#### Actividad 3:
##### TO DO ->    Para el dataframe "ratings_df":
- ##### Renombra la columna "movieId" por "movie_id"
- ##### Renombra la columna "userId" por "user_id"
- ##### Convierte la columna "timestamp" a formato TimeStampType con formato logico yyyy-MM-dd HH:mm:ss, la nueva columna generada será "time". 
    - Utiliza la función: from_unixtime -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.from_unixtime.html#pyspark.sql.functions.from_unixtime
- ##### Elimina la columna "timestamp"
##### NO UTILIZAR withColumn NI withColumnRenamed

In [67]:
## TU CODIGO VA EN ESTA CELDA:
casted_tags_df = tags_df # aplicar transformaciones a tags_df

# Importaciones extra
from pyspark.sql.functions import col, from_unixtime
from pyspark.sql.types import TimestampType

# Renombrar columnas y convertir "timestamp" a TimeStampType
# Se agregó "tag" porque no me aparecia en el dataframe resultante
casted_tags_df = casted_tags_df.select(
    col("movieId").alias("movie_id"),
    col("userId").alias("user_id"),
    col("tag"),
    (from_unixtime(col("timestamp"), "yyyy-MM-dd HH:mm:ss").cast(TimestampType()).alias("time"))
)

# Eliminar columna "timestamp"
casted_tags_df = casted_tags_df.drop("timestamp")

# Imprimir resultados
casted_tags_df.show()

+--------+-------+--------------------+-------------------+
|movie_id|user_id|                 tag|               time|
+--------+-------+--------------------+-------------------+
|     832| 224183|              acting|2017-06-05 07:20:27|
|     832| 224183|              action|2017-06-05 07:20:01|
|     832| 224183|       Brawley Nolte|2017-06-05 07:19:31|
|     832| 224183|   conspiracy theory|2017-06-05 07:20:00|
|     832| 224183|Director: Ron Howard|2017-06-05 07:15:05|
|     832| 224183|     Donnie Wahlberg|2017-06-05 07:19:33|
|     832| 224183|         Gary Sinise|2017-06-05 07:19:30|
|     832| 224183|          kidnapping|2017-06-05 07:19:57|
|     832| 224183|      Liev Schreiber|2017-06-05 07:19:36|
|     832| 224183|         Lili Taylor|2017-06-05 07:19:28|
|     832| 224183|          Mel Gibson|2017-06-05 07:19:28|
|     832| 224183|          Ron Howard|2017-06-05 07:20:26|
|     832| 224183|soundtrack: James...|2017-06-05 07:19:38|
|     832| 224183|               tense|2

In [68]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
casted_tags_df.show(1)
"""
Ejemplo de salida esperada:
+-------+--------+------+-------------------+
|user_id|movie_id|   tag|               time|
+-------+--------+------+-------------------+
| 224183|     832|acting|2017-06-05 07:20:27|
+-------+--------+------+-------------------+
only showing top 1 row
"""

+--------+-------+------+-------------------+
|movie_id|user_id|   tag|               time|
+--------+-------+------+-------------------+
|     832| 224183|acting|2017-06-05 07:20:27|
+--------+-------+------+-------------------+
only showing top 1 row



'\nEjemplo de salida esperada:\n+-------+--------+------+-------------------+\n|user_id|movie_id|   tag|               time|\n+-------+--------+------+-------------------+\n| 224183|     832|acting|2017-06-05 07:20:27|\n+-------+--------+------+-------------------+\nonly showing top 1 row\n'

In [69]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
assert "user_id" in casted_tags_df.columns
assert "movie_id" in casted_tags_df.columns
assert "tag" in casted_tags_df.columns
assert "time" in casted_tags_df.columns
assert len(casted_tags_df.columns) == 4

casted_tags_df = casted_tags_df.select("user_id" ,"movie_id", "tag", "time")

assert schema_to_ddl(casted_tags_df) == 'user_id STRING,movie_id STRING,tag STRING,time TIMESTAMP'

#### Actividad 4:
##### TO DO ->    Para el dataframe resultante de la actividad 1 (casted_movies_df) aplica la siguiente regla de negocio:
- ##### Agrega una columna llamada "year" de tipo IntegerType() de acuerdo a las siguientes condiciones:
    - ##### Si la columna "title" termina con la expresión regular "\\([0-9]{4}\\\)\\$" extraer 4 carácteres desde la posición -5
    - ##### Si la columna "title" termina con la expresión regular '\\([0-9]{4}\\\)"\\$' extraer 4 carácteres desde la posición -6
    - ##### En cualquier otro caso mantener un valor null
    - Utiliza la función substring -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.substring.html#pyspark.sql.functions.substring
    - Adicional necesitarás revisar la siguiente fucion de la clase Column: rlike -> https://spark.apache.org/docs/latest/api/python//reference/pyspark.sql/api/pyspark.sql.Column.rlike.html
        - ##### Para validar qué hace cada expresión regular puedes utilizar https://regex101.com/ o https://regexr.com/ y hacer pruebas con los textos:
            - ##### Toy Story (1995)
            - ##### """Great Performances"" Cats (1998)"
- ##### Filtra todos los registros que no contengan el valor "(no genres listed)" dentro de la columna "genres"
    - Utiliza la función  array_contains -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.array_contains.html#pyspark.sql.functions.array_contains
##### NO UTILIZAR withColumn NI withColumnRenamed

In [80]:
## TU CODIGO VA EN ESTA CELDA:
transformed_movies_df = casted_movies_df # aplicar transformaciones a casted_movies_df

# Importaciones extra
from pyspark.sql.functions import col, substring, when, array_contains
from pyspark.sql.types import IntegerType

# Agregar columna "year" de acuerdo a condiciones de expresiones regulares
transformed_movies_df = transformed_movies_df.select(
    "*",
    when(
        col("title").rlike(r"\([0-9]{4}\)$"),
        substring(col("title"), -5, 4).cast(IntegerType())
    ).when(
        col("title").rlike(r'\([0-9]{4}"$'),
        substring(col("title"), -6, 4).cast(IntegerType())
    ).otherwise(None).alias("year")
)

# Filtro de los registros
transformed_movies_df = transformed_movies_df.filter(~array_contains(col("genres"), "(no genres listed)"))

# Imprimir resultados
transformed_movies_df.show()


+--------+--------------------+--------------------+----+
|movie_id|               title|              genres|year|
+--------+--------------------+--------------------+----+
|       1|    Toy Story (1995)|[Adventure, Anima...|1995|
|       2|      Jumanji (1995)|[Adventure, Child...|1995|
|       3|Grumpier Old Men ...|   [Comedy, Romance]|1995|
|       4|Waiting to Exhale...|[Comedy, Drama, R...|1995|
|       5|Father of the Bri...|            [Comedy]|1995|
|       6|         Heat (1995)|[Action, Crime, T...|1995|
|       7|      Sabrina (1995)|   [Comedy, Romance]|1995|
|       8| Tom and Huck (1995)|[Adventure, Child...|1995|
|       9| Sudden Death (1995)|            [Action]|1995|
|      10|    GoldenEye (1995)|[Action, Adventur...|1995|
|      11|American Presiden...|[Comedy, Drama, R...|1995|
|      12|Dracula: Dead and...|    [Comedy, Horror]|1995|
|      13|        Balto (1995)|[Adventure, Anima...|1995|
|      14|        Nixon (1995)|             [Drama]|1995|
|      15|Cutt

In [81]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
transformed_movies_df.show(1)
"""
Ejemplo de salida esperada:
+--------+----------------+--------------------+----+
|movie_id|           title|              genres|year|
+--------+----------------+--------------------+----+
|       1|Toy Story (1995)|[Adventure, Anima...|1995|
+--------+----------------+--------------------+----+
only showing top 1 row
"""

+--------+----------------+--------------------+----+
|movie_id|           title|              genres|year|
+--------+----------------+--------------------+----+
|       1|Toy Story (1995)|[Adventure, Anima...|1995|
+--------+----------------+--------------------+----+
only showing top 1 row



'\nEjemplo de salida esperada:\n+--------+----------------+--------------------+----+\n|movie_id|           title|              genres|year|\n+--------+----------------+--------------------+----+\n|       1|Toy Story (1995)|[Adventure, Anima...|1995|\n+--------+----------------+--------------------+----+\nonly showing top 1 row\n'

In [82]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
assert "movie_id" in transformed_movies_df.columns
assert "title" in transformed_movies_df.columns
assert "genres" in transformed_movies_df.columns
assert "year" in transformed_movies_df.columns
assert len(transformed_movies_df.columns) == 4

transformed_movies_df = transformed_movies_df.select("movie_id" ,"title", "genres", "year")

assert schema_to_ddl(transformed_movies_df) == 'movie_id STRING,title STRING,genres ARRAY<STRING>,year INT'

assert transformed_movies_df.count() == 79477

In [83]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
dfs = [(transformed_movies_df, "06/movies"),
       (casted_tags_df, "06/tags"),
       (casted_ratings_df, "06/ratings")]

write_tmp_df(dfs)